In [1]:
import os
os.chdir('../py')

import pandas as pd
import numpy as np
import feather

from glob import glob

import matplotlib.pyplot as plt
import seaborn as sns
plt.rcParams["patch.force_edgecolor"] = False
plt.rcParams['font.family'] = 'Ricty Diminised'
sns.set(style="whitegrid", font='Ricty Diminised', palette="muted", color_codes=True, rc={'grid.linestyle': '--'})
red = sns.xkcd_rgb["light red"]
green = sns.xkcd_rgb["medium green"]
blue = sns.xkcd_rgb["denim blue"]

import japanize_matplotlib

import lightgbm as lgb

from sklearn.cluster import KMeans
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from sklearn.preprocessing import LabelEncoder

# my modules
from EDA import *
import utils

/anaconda3/lib/python3.7/site-packages/lightgbm/__init__.py:46: UserWarning: Starting from version 2.2.1, the library file in distribution wheels for macOS is built by the Apple Clang (Xcode_8.3.3) compiler.
This means that in case of installing LightGBM from PyPI via the ``pip install lightgbm`` command, you don't need to install the gcc compiler anymore.
Instead of that, you need to install the OpenMP library, which is required for running LightGBM on the system with the Apple Clang compiler.
You can install the OpenMP library by the following command: ``brew install libomp``.
  "You can install the OpenMP library by the following command: ``brew install libomp``.", UserWarning)


In [2]:
os.listdir('../input')

['test_transaction.csv.zip',
 'train_transaction.csv.zip',
 'train_identity.csv.zip',
 'sample_submission.csv.zip',
 'test_identity.csv.zip']

In [3]:
DIR = '../input/'

In [9]:
tr_tran = pd.read_csv(DIR + 'train_transaction.csv.zip')
tr_iden = pd.read_csv(DIR + 'train_identity.csv.zip')

te_tran = pd.read_csv(DIR + 'test_transaction.csv.zip')
te_iden = pd.read_csv(DIR + 'test_identity.csv.zip')

In [14]:
print(tr_tran.shape)
max_disp(tr_tran.head(), cols=1)

(590540, 394)


,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,credit,315.0,87.0,19.0,NaN,NaN,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,2.0,0.0,1.0,1.0,14.0,NaN,13.0,NaN,NaN,NaN,NaN,NaN,NaN,13.0,13.0,NaN,NaN,NaN,0.0,T,T,T,M2,F,T,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,117.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,117.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,credit,325.0,87.0,NaN,NaN,gmail.com,NaN,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,NaN,NaN,0.0,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,M0,T,T,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,

In [15]:
print(tr_iden.shape)
max_disp(tr_iden.head(), cols=1)

(144233, 41)


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,2987004,0.0,70787.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,100.0,NotFound,NaN,-480.0,New,NotFound,166.0,NaN,542.0,144.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,Android 7.0,samsung browser 6.2,32.0,2220x1080,match_status:2,T,F,T,T,mobile,SAMSUNG SM-G892A Build/NRD90M
1,2987008,-5.0,98945.0,NaN,NaN,0.0,-5.0,NaN,NaN,NaN,NaN,100.0,NotFound,49.0,-300.0,New,NotFound,166.0,NaN,621.0,500.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,iOS 11.1.2,mobile safari 11.0,32.0,1334x750,match_status:1,T,F,F,T,mobile,iOS Device
2,2987010,-5.0,191631.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,52.0,NaN,Found,Found,121.0,NaN,410.0,142.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,Windows
3,2987011,-5.0,221832.0,NaN,NaN,0.0,-6.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,NaN,New,NotFound,225.0,NaN,176.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 62.0,NaN,NaN,NaN,F,F,T,T,desktop,NaN
4,2987016,0.0,7460.0,0.0,0.0,1.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,NaN,-300.0,Found,Found,166.0,15.0,529.0,575.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Mac OS X 10_11_6,chrome 62.0,24.0,1280x800,match_status:2,T,F,T,T,desktop,MacOS


In [16]:
print(te_tran.shape)
max_disp(te_tran.head(), cols=1)

(506691, 393)


,TransactionID,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,card6,addr1,addr2,dist1,dist2,P_emaildomain,R_emaildomain,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,C12,C13,C14,D1,D2,D3,D4,D5,D6,D7,D8,D9,D10,D11,D12,D13,D14,D15,M1,M2,M3,M4,M5,M6,M7,M8,M9,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35,V36,V37,V38,V39,V40,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50,V51,V52,V53,V54,V55,V56,V57,V58,V59,V60,V61,V62,V63,V64,V65,V66,V67,V68,V69,V70,V71,V72,V73,V74,V75,V76,V77,V78,V79,V80,V81,V82,V83,V84,V85,V86,V87,V88,V89,V90,V91,V92,V93,V94,V95,V96,V97,V98,V99,V100,V101,V102,V103,V104,V105,V106,V107,V108,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118,V119,V120,V121,V122,V123,V124,V125,V126,V127,V128,V129,V130,V131,V132,V133,V134,V135,V136,V137,V138,V139,V140,V141,V142,V143,V144,V145,V146,V147,V148,V149,V150,V151,V152,V153,V154,V155,V156,V157,V158,V159,V160,V161,V162,V163,V164,V165,V166,V167,V168,V169,V170,V171,V172,V173,V174,V175,V176,V177,V178,V179,V180,V181,V182,V183,V184,V185,V186,V187,V188,V189,V190,V191,V192,V193,V194,V195,V196,V197,V198,V199,V200,V201,V202,V203,V204,V205,V206,V207,V208,V209,V210,V211,V212,V213,V214,V215,V216,V217,V218,V219,V220,V221,V222,V223,V224,V225,V226,V227,V228,V229,V230,V231,V232,V233,V234,V235,V236,V237,V238,V239,V240,V241,V242,V243,V244,V245,V246,V247,V248,V249,V250,V251,V252,V253,V254,V255,V256,V257,V258,V259,V260,V261,V262,V263,V264,V265,V266,V267,V268,V269,V270,V271,V272,V273,V274,V275,V276,V277,V278,V279,V280,V281,V282,V283,V284,V285,V286,V287,V288,V289,V290,V291,V292,V293,V294,V295,V296,V297,V298,V299,V300,V301,V302,V303,V304,V305,V306,V307,V308,V309,V310,V311,V312,V313,V314,V315,V316,V317,V318,V319,V320,V321,V322,V323,V324,V325,V326,V327,V328,V329,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,3663549,18403224,31.95,W,10409,111.0,150.0,visa,226.0,debit,170.0,87.0,1.0,NaN,gmail.com,NaN,6.0,6.0,0.0,0.0,3.0,4.0,0.0,0.0,6.0,0.0,5.0,1.0,115.0,6.0,419.0,419.0,27.0,398.0,27.0,NaN,NaN,NaN,NaN,418.0,203.0,NaN,NaN,NaN,409.0,T,T,F,NaN,NaN,F,T,T,T,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.000000,47.950001,0.000000,0.000000,47.950001,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,3663550,18403263,49.00,W,4272,111.0,150.0,visa,226.0,debit,299.0,87.0,4.0,NaN,aol.com,NaN,3.0,2.0,0.0,0.0,0.0,1.0,0.0,0.0,2.0,0.0,1.0,1.0,12.0,2.0,149.0,149.0,7.0,634.0,7.0,NaN,NaN,NaN,NaN,231.0,634.0,NaN,NaN,NaN,634.0,T,F,F,M0,NaN,F,NaN,NaN,NaN,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0

In [18]:
print(te_iden.shape)
max_disp(te_iden.head(), cols=1)

(141907, 41)


,TransactionID,id_01,id_02,id_03,id_04,id_05,id_06,id_07,id_08,id_09,id_10,id_11,id_12,id_13,id_14,id_15,id_16,id_17,id_18,id_19,id_20,id_21,id_22,id_23,id_24,id_25,id_26,id_27,id_28,id_29,id_30,id_31,id_32,id_33,id_34,id_35,id_36,id_37,id_38,DeviceType,DeviceInfo
0,3663586,-45.0,280290.0,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
1,3663588,0.0,3579.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,Found,NaN,-300.0,Found,Found,166.0,NaN,542.0,368.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,Android 6.0.1,chrome 67.0 for android,24.0,1280x720,match_status:2,T,F,T,T,mobile,LGLS676 Build/MXB48T
2,3663597,-5.0,185210.0,NaN,NaN,1.0,0.0,NaN,NaN,NaN,NaN,100.0,NotFound,52.0,-360.0,New,NotFound,225.0,NaN,271.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,ie 11.0 for tablet,NaN,NaN,NaN,F,T,T,F,desktop,Trident/7.0
3,3663601,-45.0,252944.0,0.0,0.0,0.0,0.0,NaN,NaN,0.0,0.0,100.0,NotFound,27.0,NaN,Found,Found,225.0,15.0,427.0,563.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Found,Found,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,MYA-L13 Build/HUAWEIMYA-L13
4,3663602,-95.0,328680.0,NaN,NaN,7.0,-33.0,NaN,NaN,NaN,NaN,100.0,NotFound,27.0,NaN,New,NotFound,225.0,15.0,567.0,507.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,New,NotFound,NaN,chrome 67.0 for android,NaN,NaN,NaN,F,F,T,F,mobile,SM-G9650 Build/R16NW


In [19]:
info_tr_tran = get_info(tr_tran)
info_tr_iden = get_info(tr_iden)
info_te_tran = get_info(te_tran)
info_te_iden = get_info(te_iden)

Shape: (590540, 394)
Shape: (144233, 41)
Shape: (506691, 393)
Shape: (141907, 41)


In [21]:
max_disp(info_tr_tran, rows=1)

,DataType,Nulls,Null%,Uniques,Unique%,Min,Mean,Max,Std,top10 val,top10 cnt,top10 raito
TransactionID,int64,0,0.000000,590540,100.000000,2987000.000,3.282270e+06,3.577539e+06,1.704744e+05,"[3147775, 3089230, 3189951, 3187902, 3194045, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
isFraud,int64,0,0.000000,2,0.000339,0.000,3.499001e-02,1.000000e+00,1.837546e-01,"[0, 1]","[569877, 20663]","[0.97, 0.03]"
TransactionDT,int64,0,0.000000,573349,97.088936,86400.000,7.372311e+06,1.581113e+07,4.617224e+06,"[9474817, 11576951, 7236588, 4397066, 8468062,...","[8, 5, 5, 5, 4, 4, 4, 4, 4, 4]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
TransactionAmt,float64,0,0.000000,20902,3.539472,0.251,1.350272e+02,3.193739e+04,2.391625e+02,"[59.0, 117.0, 107.95, 57.95, 100.0, 50.0, 49.0...","[30582, 28933, 23954, 23600, 20362, 17009, 159...","[0.05, 0.05, 0.04, 0.04, 0.03, 0.03, 0.03, 0.0..."
ProductCD,object,0,0.000000,5,0.000847,NaN,NaN,NaN,NaN,"['W', 'C', 'R', 'H', 'S']","[439670, 68519, 37699, 33024, 11628]","[0.74, 0.12, 0.06, 0.06, 0.02]"
card1,int64,0,0.000000,13553,2.295018,1000.000,9.898735e+03,1.839600e+04,4.901170e+03,"[7919, 9500, 15885, 17188, 15066, 12695, 12544...","[14932, 14162, 10361, 10344, 7945, 7091, 6773,...","[0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.0..."
card2,float64,8933,1.512683,500,0.084668,100.000,3.625555e+02,6.000000e+02,1.577932e+02,"[321.0, 111.0, 555.0, 490.0, 583.0, 170.0, 194...","[48935, 45191, 41995, 38145, 21803, 18214, 169...","[0.08, 0.08, 0.07, 0.06, 0.04, 0.03, 0.03, 0.0..."
card3,float64,1565,0.265012,114,0.019304,100.000,1.531949e+02,2.310000e+02,1.133644e+01,"[150.0, 185.0, 106.0, 146.0, 144.0, 117.0, 143...","[521287, 56346, 1571, 1252, 1252, 962, 899, 75...","[0.88, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
card4,object,1577,0.267044,4,0.000677,NaN,NaN,NaN,NaN,"['visa', 'mastercard', 'american express', 'di...","[384767, 189217, 8328, 6651]","[0.65, 0.32, 0.01, 0.01]"
card5,float64,4259,0.721204,119,0.020151,100.000,1.992789e+02,2.370000e+02,4.124445e+01,"[226.0, 224.0, 166.0, 102.0, 117.0, 138.0, 195...","[296546, 81513, 57140, 29105, 25941, 19737, 16...","[0.5, 0.14, 0.1, 0.05, 0.04, 0.03, 0.03, 0.02,..."


In [22]:
max_disp(info_tr_iden, rows=1)

,DataType,Nulls,Null%,Uniques,Unique%,Min,Mean,Max,Std,top10 val,top10 cnt,top10 raito
TransactionID,int64,0,0.000000,144233,100.000000,2987004.0,3.236329e+06,3577534.0,178849.571186,"[3153916, 3548654, 3079669, 3077620, 3067379, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_01,float64,0,0.000000,77,0.053386,-100.0,-1.017050e+01,0.0,14.347949,"[-5.0, 0.0, -10.0, -20.0, -15.0, -25.0, -45.0,...","[82170, 19555, 11257, 11211, 5674, 4623, 2143,...","[0.57, 0.14, 0.08, 0.08, 0.04, 0.03, 0.01, 0.0..."
id_02,float64,3361,2.330257,115655,80.186226,1.0,1.747166e+05,999595.0,159651.816856,"[1102.0, 696.0, 1116.0, 1117.0, 1120.0, 1141.0...","[11, 10, 9, 9, 9, 9, 9, 8, 7, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_03,float64,77909,54.016071,24,0.016640,-13.0,6.018937e-02,10.0,0.598231,"[0.0, 1.0, 3.0, 2.0, 5.0, 4.0, 6.0, -5.0, -6.0...","[63903, 863, 668, 421, 109, 100, 64, 33, 31, 21]","[0.44, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
id_04,float64,77909,54.016071,15,0.010400,-28.0,-5.893794e-02,0.0,0.701015,"[0.0, -5.0, -6.0, -8.0, -4.0, -1.0, -11.0, -12...","[65739, 132, 98, 64, 51, 43, 35, 34, 30, 26]","[0.46, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
id_05,float64,7368,5.108401,93,0.064479,-72.0,1.615585e+00,52.0,5.249856,"[0.0, 1.0, 2.0, 3.0, 4.0, 6.0, 5.0, 7.0, 8.0, ...","[92743, 8293, 4937, 3854, 2941, 2338, 2312, 14...","[0.64, 0.06, 0.03, 0.03, 0.02, 0.02, 0.02, 0.0..."
id_06,float64,7368,5.108401,101,0.070026,-100.0,-6.698710e+00,0.0,16.491104,"[0.0, -1.0, -5.0, -6.0, -9.0, -100.0, -8.0, -1...","[91325, 4687, 3849, 3257, 2634, 2431, 1994, 19...","[0.63, 0.03, 0.03, 0.02, 0.02, 0.02, 0.01, 0.0..."
id_07,float64,139078,96.425922,84,0.058239,-46.0,1.328535e+01,61.0,11.384207,"[0.0, 16.0, 14.0, 12.0, 15.0, 17.0, 4.0, 13.0,...","[409, 245, 228, 208, 186, 180, 178, 175, 154, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_08,float64,139078,96.425922,94,0.065172,-100.0,-3.860039e+01,0.0,26.084899,"[-100.0, 0.0, -34.0, -33.0, -32.0, -35.0, -25....","[500, 261, 257, 209, 185, 184, 126, 117, 116, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_09,float64,69307,48.052110,46,0.031893,-36.0,9.102314e-02,25.0,0.983842,"[0.0, 1.0, 3.0, 2.0, 4.0, 5.0, 6.0, -6.0, -5.0...","[70378, 1616, 966, 773, 270, 207, 145, 66, 60,...","[0.49, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0..."


In [23]:
max_disp(info_te_tran, rows=1)

,DataType,Nulls,Null%,Uniques,Unique%,Min,Mean,Max,Std,top10 val,top10 cnt,top10 raito
TransactionID,int64,0,0.000000,506691,100.000000,3.663549e+06,3.916894e+06,4.170239e+06,1.462692e+05,"[3663619, 3870155, 3882445, 3876302, 3878351, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
TransactionDT,int64,0,0.000000,494686,97.630706,1.840322e+07,2.692994e+07,3.421434e+07,4.756507e+06,"[33613413, 26576684, 33750653, 19601418, 32146...","[4, 4, 4, 4, 4, 4, 3, 3, 3, 3]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
TransactionAmt,float64,0,0.000000,14119,2.786511,1.800000e-02,1.347256e+02,1.027000e+04,2.457798e+02,"[59.0, 117.0, 107.95, 57.95, 100.0, 50.0, 49.0...","[27365, 25798, 20243, 20063, 17565, 14469, 142...","[0.05, 0.05, 0.04, 0.04, 0.03, 0.03, 0.03, 0.0..."
ProductCD,object,0,0.000000,5,0.000987,NaN,NaN,NaN,NaN,"['W', 'C', 'R', 'H', 'S']","[360987, 69266, 35647, 29373, 11418]","[0.71, 0.14, 0.07, 0.06, 0.02]"
card1,int64,0,0.000000,13244,2.613822,1.001000e+03,9.957222e+03,1.839700e+04,4.884961e+03,"[7919, 15885, 9500, 17188, 15066, 6019, 12544,...","[13083, 12330, 12081, 9262, 6661, 6497, 5921, ...","[0.03, 0.02, 0.02, 0.02, 0.01, 0.01, 0.01, 0.0..."
card2,float64,8654,1.707944,497,0.098087,1.000000e+02,3.637354e+02,6.000000e+02,1.586887e+02,"[321.0, 555.0, 111.0, 490.0, 583.0, 545.0, 170...","[42796, 38409, 37346, 32351, 19700, 15539, 151...","[0.08, 0.08, 0.07, 0.06, 0.04, 0.03, 0.03, 0.0..."
card3,float64,3002,0.592472,115,0.022696,1.000000e+02,1.535434e+02,2.320000e+02,1.244301e+01,"[150.0, 185.0, 106.0, 117.0, 144.0, 146.0, 143...","[435558, 53614, 1972, 1879, 1381, 1218, 771, 7...","[0.86, 0.11, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0..."
card4,object,3086,0.609050,4,0.000789,NaN,NaN,NaN,NaN,"['visa', 'mastercard', 'american express', 'di...","[334882, 158169, 7681, 2873]","[0.66, 0.31, 0.02, 0.01]"
card5,float64,4547,0.897391,102,0.020131,1.000000e+02,2.001630e+02,2.370000e+02,4.056246e+01,"[226.0, 224.0, 166.0, 138.0, 117.0, 102.0, 195...","[256991, 71596, 45790, 22102, 21120, 20386, 14...","[0.51, 0.14, 0.09, 0.04, 0.04, 0.04, 0.03, 0.0..."
card6,object,3007,0.593458,3,0.000592,NaN,NaN,NaN,NaN,"['debit', 'credit', 'charge card']","[385021, 118662, 1]","[0.76, 0.23, 0.0]"


In [24]:
max_disp(info_te_iden, rows=1)

,DataType,Nulls,Null%,Uniques,Unique%,Min,Mean,Max,Std,top10 val,top10 cnt,top10 raito
TransactionID,int64,0,0.000000,141907,100.000000,3663586.0,3.972166e+06,4170239.0,146996.579622,"[4162786, 3989984, 3943727, 3945772, 3695910, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_01,float64,0,0.000000,80,0.056375,-100.0,-1.132573e+01,0.0,14.508520,"[-5.0, 0.0, -10.0, -20.0, -15.0, -25.0, -45.0,...","[82579, 12000, 11793, 11130, 7048, 5017, 4552,...","[0.58, 0.08, 0.08, 0.08, 0.05, 0.04, 0.03, 0.0..."
id_02,float64,4931,3.474811,114868,80.945972,2.0,1.926587e+05,999869.0,182613.277215,"[312.0, 552.0, 1203.0, 911.0, 309.0, 701.0, 28...","[9, 9, 8, 8, 8, 8, 7, 7, 7, 7]","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_03,float64,75426,53.151712,22,0.015503,-12.0,5.300763e-02,11.0,0.684551,"[0.0, 1.0, 3.0, 2.0, 5.0, 4.0, 6.0, -5.0, -6.0...","[63908, 782, 774, 385, 155, 84, 70, 55, 53, 46]","[0.45, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0.0, 0...."
id_04,float64,75426,53.151712,15,0.010570,-19.0,-8.745356e-02,0.0,0.840351,"[0.0, -6.0, -5.0, -9.0, -12.0, -7.0, -11.0, -4...","[65651, 199, 185, 107, 77, 46, 40, 37, 36, 33]","[0.46, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0,..."
id_05,float64,7157,5.043444,90,0.063422,-81.0,1.246033e+00,52.0,5.071394,"[0.0, 1.0, 2.0, 3.0, 4.0, 5.0, 6.0, 7.0, 8.0, ...","[90016, 7868, 4711, 3641, 2783, 2236, 1915, 14...","[0.63, 0.06, 0.03, 0.03, 0.02, 0.02, 0.01, 0.0..."
id_06,float64,7157,5.043444,101,0.071173,-100.0,-6.803829e+00,0.0,15.921457,"[0.0, -1.0, -5.0, -6.0, -9.0, -11.0, -100.0, -...","[84268, 7207, 3774, 3453, 2592, 2312, 2154, 21...","[0.59, 0.05, 0.03, 0.02, 0.02, 0.02, 0.02, 0.0..."
id_07,float64,136848,96.434989,81,0.057080,-41.0,1.249318e+01,59.0,11.678206,"[0.0, 12.0, 13.0, 2.0, 15.0, 3.0, 14.0, 1.0, 5...","[527, 191, 189, 181, 164, 160, 156, 152, 152, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_08,float64,136848,96.434989,90,0.063422,-100.0,-3.657778e+01,0.0,25.544185,"[-100.0, 0.0, -34.0, -33.0, -35.0, -31.0, -36....","[443, 342, 301, 246, 217, 177, 137, 136, 123, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
id_09,float64,67569,47.614987,38,0.026778,-32.0,7.621943e-02,16.0,1.009687,"[0.0, 1.0, 3.0, 2.0, 5.0, 4.0, 6.0, -4.0, -5.0...","[70014, 1381, 1008, 731, 243, 180, 140, 96, 88...","[0.49, 0.01, 0.01, 0.01, 0.0, 0.0, 0.0, 0.0, 0..."
